In [1]:
import sys
sys.path.append("..")

import pandas as pd
from src.model_training_utils import load_data_columns_config, load_metrics
from src.model_analysis import summarize_metrics

config_dict = load_data_columns_config(version=2)
factors_columns = list(config_dict.keys())[1:-1]

# Specify the custom order for the 'factor_type'
custom_order = ['train', 'validation', 'test']

GPU devices are already configured, skipping setup.


#### [1]. Matrics

In [2]:
### v2 metrics: predictor enhancement excluding industry factors
factors_columns = ['tech_factors',
 'calendar_factors',
 'fundamental_factors',
 'industry_factors',
 'release_schedule_factors']
multi_idx = pd.MultiIndex.from_product([factors_columns, custom_order], names=['factor_type', 'type'])

filename_roots=("train_metrics_dict_lstm_single_cate_v2_", "test_metrics_lstm_single_cate_v2_")
lstm_metrics_summary = summarize_metrics(filename_roots, factors_columns)

filename_roots=("train_metrics_dict_lstm_v2_", "test_metrics_lstm_v2_")
lstm_cum_metrics_summary = summarize_metrics(filename_roots, factors_columns)

In [3]:
lstm_summary_df = lstm_metrics_summary.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_summary_df = lstm_summary_df.reindex(multi_idx)
lstm_cum_summary_df = lstm_cum_metrics_summary.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_cum_summary_df = lstm_cum_summary_df.reindex(multi_idx)
pd.merge(lstm_summary_df, lstm_cum_summary_df, left_index=True, right_index=True,
         suffixes=("_single","_cum"))

MSE_single  r2_single   MSE_cum    r2_cum
factor_type              type                                                 
tech_factors             train         0.188753   0.932351  0.188753  0.932351
                         validation    0.191848   0.927121  0.191848  0.927121
                         test          0.177148   0.930494  0.177148  0.930494
calendar_factors         train         2.928466  -0.017266  0.194635  0.930364
                         validation    2.813014  -0.036089  0.212802  0.919796
                         test          2.694860  -0.039661  0.191246  0.925109
fundamental_factors      train         0.337659   0.880328  0.185809  0.933783
                         validation    0.633209   0.762445  0.195223  0.926320
                         test          1.076532   0.583168  0.182545  0.928689
industry_factors         train         1.523516   0.467135  0.186221  0.933648
                         validation    1.694847   0.370618  0.196352  0.925799
                         test          2.121662   0.179763  0.186856  0.926998
release_schedule_factors train         2.924034  -0.016128  0.179460  0.935859
                         validation    2.762343  -0.017900  0.189331  0.928230
                         test          2.666896  -0.028936  0.176203  0.930987

In [4]:
### Version 3: predictor enhancement including industry factors: aggregation in industry
factors_columns = ['tech_factors',
 'calendar_factors',
 'fundamental_factors',
 'agg_industry_factors',
 'release_schedule_factors']
# single
filename_roots=("train_metrics_dict_lstm_single_cate_v3_", "test_metrics_lstm_single_cate_v3_")
lstm_metrics_summary = summarize_metrics(filename_roots, factors_columns, [4])
lstm_summary_df = lstm_metrics_summary.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_summary_df.reindex(pd.MultiIndex.from_product([["agg_industry_factors"], custom_order], names=['factor_type', 'type']))

MSE        r2
factor_type          type                          
agg_industry_factors train       2.214445  0.226653
                     validation  2.192936  0.188228
                     test        2.111926  0.184501

In [5]:
# cumulative
filename_roots=("train_metrics_dict_lstm_v3_", "test_metrics_lstm_v3_")
lstm_cum_metrics_summary = summarize_metrics(filename_roots, factors_columns, [4, 5])
lstm_cum_summary_df = lstm_cum_metrics_summary.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_cum_summary_df.reindex(pd.MultiIndex.from_product([["agg_industry_factors", "release_schedule_factors"], custom_order], names=['factor_type', 'type']))

MSE        r2
factor_type              type                          
agg_industry_factors     train       0.191401  0.931824
                         validation  0.196997  0.925537
                         test        0.192868  0.924618
release_schedule_factors train       0.199659  0.928330
                         validation  0.352938  0.866806
                         test        0.190422  0.925384

In [2]:
### Version 4: autoencoder trianable + naive industry
factors_columns = ['tech_factors',
 'calendar_factors',
 'fundamental_factors',
 'industry_factors',
 'release_schedule_factors']

# cumulative
# I discarded the v4 data. So, I rerun it with v6 tab
filename_roots=("train_metrics_dict_lstm_l1_v6_ae_", "test_metrics_lstm_l1_v6_ae_")
lstm_cum_metrics_summary = summarize_metrics(filename_roots, factors_columns, [3, 4, 5])
lstm_cum_summary_df = lstm_cum_metrics_summary.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_cum_summary_df.reindex(pd.MultiIndex.from_product([factors_columns[2:], custom_order], names=['factor_type', 'type']))

MSE        r2
factor_type              type                          
fundamental_factors      train       0.183675  0.934573
                         validation  0.192026  0.927555
                         test        0.178453  0.930197
industry_factors         train       0.183840  0.934481
                         validation  0.190836  0.927950
                         test        0.176661  0.930887
release_schedule_factors train       0.182792  0.934832
                         validation  0.197040  0.925002
                         test        0.180737  0.929064

In [11]:
### Version 6: autoencoder trianable + naive industry + predictor enhancement excluding industry factors
# Other comparison:
# trainable vs. non-trainable
# lstm_l1_3_ae_test3_v6: tech+fun (encoder non-trainable)
# lstm_l1_3_ae_test4_v6: tech+fun/ encoder trainable

# non-trainable
filename_roots=("train_metrics_dict_lstm_l1_v6_ae_test3_", "test_metrics_lstm_l1_v6_ae_test3_")
lstm_cum_metrics_summary1 = summarize_metrics(filename_roots, factors_columns, [3])
lstm_cum_summary_df = lstm_cum_metrics_summary1.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_cum_summary_df_std = lstm_cum_metrics_summary1.groupby(["factor_type", "type"]).std().drop(columns=["cv_num"])
index=pd.MultiIndex.from_product([factors_columns[2:3], custom_order], names=['factor_type', 'type'])

final_df = pd.concat([lstm_cum_summary_df, lstm_cum_summary_df_std], keys=['mean', 'std'], axis=1).reindex(index)
final_df.columns = ['_'.join(col) for col in final_df.columns]
final_df

mean_MSE   mean_r2   std_MSE    std_r2
factor_type         type                                              
fundamental_factors train       0.190037  0.931896  0.008744  0.006008
                    validation  0.193529  0.926461  0.025715  0.009683
                    test        0.178861  0.929843  0.006469  0.002604

In [13]:
# trainable
filename_roots=("train_metrics_dict_lstm_l1_v6_ae_test4_", "test_metrics_lstm_l1_v6_ae_test4_")
lstm_cum_metrics_summary2 = summarize_metrics(filename_roots, factors_columns, [3])
lstm_cum_summary_df = lstm_cum_metrics_summary2.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_cum_summary_df_std = lstm_cum_metrics_summary2.groupby(["factor_type", "type"]).std().drop(columns=["cv_num"])
index=pd.MultiIndex.from_product([factors_columns[2:3], custom_order], names=['factor_type', 'type'])

final_df = pd.concat([lstm_cum_summary_df, lstm_cum_summary_df_std], keys=['mean', 'std'], axis=1).reindex(index)
final_df.columns = ['_'.join(col) for col in final_df.columns]
final_df

mean_MSE   mean_r2   std_MSE    std_r2
factor_type         type                                              
fundamental_factors train       0.188397  0.932492  0.008016  0.005876
                    validation  0.193984  0.926247  0.025275  0.009608
                    test        0.180179  0.929329  0.007602  0.003047

In [14]:
# compare trainable vs. non-trainable
concat_df = lstm_cum_metrics_summary1.merge(lstm_cum_metrics_summary2, on=["type", "cv_num", "factor_type"],
                               suffixes=("_nontrainable","_trainable"))
# 1. train:
concat_df[concat_df["type"]=="train"]

,type,MSE_nontrainable,r2_nontrainable,cv_num,factor_type,MSE_trainable,r2_trainable
1,train,0.175733,0.941671,1,fundamental_factors,0.175019,0.941904
4,train,0.194248,0.934397,2,fundamental_factors,0.191556,0.935307
7,train,0.194460,0.933838,3,fundamental_factors,0.193232,0.934266
10,train,0.200935,0.927438,4,fundamental_factors,0.198043,0.928542
13,train,0.185427,0.928931,5,fundamental_factors,0.184423,0.929319
16,train,0.189419,0.925097,6,fundamental_factors,0.188106,0.925612


In [15]:
# 2. test:
concat_df[concat_df["type"]=="test"]

,type,MSE_nontrainable,r2_nontrainable,cv_num,factor_type,MSE_trainable,r2_trainable
0,test,0.192011,0.924549,1,fundamental_factors,0.195684,0.923116
3,test,0.176855,0.930665,2,fundamental_factors,0.176901,0.930652
6,test,0.175754,0.931065,3,fundamental_factors,0.177245,0.930471
9,test,0.176431,0.930823,4,fundamental_factors,0.177277,0.930496
12,test,0.175342,0.931269,5,fundamental_factors,0.176553,0.930805
15,test,0.176775,0.930684,6,fundamental_factors,0.177414,0.930435


In [20]:
# Add different sets of factors
# lstm_l1_3_ae_test4_v6: tech+fun/ encoder trainable
# lstm_l1_3_ae_test2_v6: tech+cal+fun/ encoder trainable
# lstm_l1_3_ae_test5_v6: tech+cal+fun+release/ encoder trainable

filename_roots=("train_metrics_dict_lstm_l1_v6_ae_test4_", "test_metrics_lstm_l1_v6_ae_test4_")
lstm_cum_metrics_summary1 = summarize_metrics(filename_roots, factors_columns, [3])
lstm_cum_summary_df1 = lstm_cum_metrics_summary1.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_cum_summary_df_std1 = lstm_cum_metrics_summary1.groupby(["factor_type", "type"]).std().drop(columns=["cv_num"])

filename_roots=("train_metrics_dict_lstm_l1_v6_ae_test2_", "test_metrics_lstm_l1_v6_ae_test2_")
lstm_cum_metrics_summary2 = summarize_metrics(filename_roots, factors_columns, [3])
lstm_cum_summary_df2 = lstm_cum_metrics_summary2.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_cum_summary_df_std2 = lstm_cum_metrics_summary2.groupby(["factor_type", "type"]).std().drop(columns=["cv_num"])

filename_roots=("train_metrics_dict_lstm_l1_v6_ae_test5_", "test_metrics_lstm_l1_v6_ae_test5_")
lstm_cum_metrics_summary3 = summarize_metrics(filename_roots, factors_columns, [3])
lstm_cum_summary_df3 = lstm_cum_metrics_summary3.groupby(["factor_type", "type"]).mean().drop(columns=["cv_num"])
lstm_cum_summary_df_std3 = lstm_cum_metrics_summary3.groupby(["factor_type", "type"]).std().drop(columns=["cv_num"])

index=pd.MultiIndex.from_product([factors_columns[2:3], custom_order], names=['factor_type', 'type'])

# lstm_l1_3_ae_test4_v6: tech+fun/ encoder trainable
final_df = pd.concat([lstm_cum_summary_df1, lstm_cum_summary_df_std1], keys=['mean', 'std'], axis=1).reindex(index)
final_df.columns = ['_'.join(col) for col in final_df.columns]
final_df

mean_MSE   mean_r2   std_MSE    std_r2
factor_type         type                                              
fundamental_factors train       0.188397  0.932492  0.008016  0.005876
                    validation  0.193984  0.926247  0.025275  0.009608
                    test        0.180179  0.929329  0.007602  0.003047

In [17]:
# lstm_l1_3_ae_test2_v6: tech+cal+fun/ encoder trainable
final_df2 = pd.concat([lstm_cum_summary_df2, lstm_cum_summary_df_std2], keys=['mean', 'std'], axis=1).reindex(index)
final_df2.columns = ['_'.join(col) for col in final_df2.columns]
final_df2

mean_MSE   mean_r2   std_MSE    std_r2
factor_type         type                                              
fundamental_factors train       0.184442  0.934300  0.006629  0.005104
                    validation  0.195411  0.926332  0.035012  0.011843
                    test        0.178686  0.930121  0.008157  0.003207

In [18]:
# lstm_l1_3_ae_test5_v6: tech+cal+fun+release/ encoder trainable
final_df3 = pd.concat([lstm_cum_summary_df3, lstm_cum_summary_df_std3], keys=['mean', 'std'], axis=1).reindex(index)
final_df3.columns = ['_'.join(col) for col in final_df3.columns]
final_df3

mean_MSE   mean_r2   std_MSE    std_r2
factor_type         type                                              
fundamental_factors train       0.177302  0.936667  0.006002  0.005018
                    validation  0.182525  0.930840  0.024109  0.009725
                    test        0.168729  0.933793  0.006124  0.002482